## Part I : scrape the Wikipedia page and preprocessing

In [1]:
import numpy as np
import pandas as pd

Obtain the dataframe from wikepedia page

In [2]:
df=pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")[0]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Drop rows where Borough == 'Not assigned'

In [31]:
df_droped = df[df.Borough != 'Not assigned'].reset_index(drop = True)
df_droped.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Downtown Toronto,Queen's Park
6,M9A,Etobicoke,Islington Avenue
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


Merge Neighbourhood with same Postcode

In [47]:
df_merged = df_droped.groupby(['Postcode','Borough'],sort=False)['Neighbourhood'].apply(', '.join).reset_index()
df_merged.rename(columns={"Postcode": "Postalcode"}, inplace=True)
df_merged.head(10)

,Postalcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [33]:
df_merged.shape

(103, 3)

## Part II: Get the latitude and the longitude coordinates of each neighborhood.

In [34]:
import geocoder 

ModuleNotFoundError: No module named 'geocoder'

In [36]:
!pip install geocoder
import geocoder

In [40]:
# def get_coord(postal_code):
#     # initialize your variable to None
#     lat_lng_coords = None
#     # loop until you get the coordinates
#     while(lat_lng_coords is None):
#         g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
#         lat_lng_coords = g.latlng
#     return lat_lng_coords

In [ ]:
# postal_codes = df['Postcode'] 
# coords = [get_coord(postal_code) for postal_code in postal_codes.tolist() ]
# df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
# df['Latitude'] = df_coords['Latitude']
# df['Longitude'] = df_coords['Longitude']
# df.head()

I have tried the geocoder, but it has no response for a long time, so let's read the csv file directly.

In [61]:
#read csv file to get the Latitude and Longitude
geo_df = pd.read_csv('http://cocl.us/Geospatial_data')
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [60]:
#Merge the two dataframes depends on the PostalCode
df_final = df_merged.merge(geo_df, how='inner', left_on='Postalcode', right_on='Postal Code')
df_final = df_final.drop('Postal Code', axis=1)
df_final.head()

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
